# Masks

With gdsfactory you can easily go from components, to sweeps, to Masks. 

Lets start with a resistance sweep, where you change the resistance width to measure sheet resistance

## Pack

In [ ]:
import gdsfactory as gf

gf.CONF.plotter = 'matplotlib'  # This notebook rendered with 'holoviews' exceeds the 100MB limit for github pages
gf.clear_cache()

sweep = [gf.components.resistance_sheet(width=width) for width in [1, 10, 100]]
m = gf.pack(sweep)
c = m[0]
c.plot()

In [ ]:
spiral_te = gf.routing.add_fiber_single(gf.functions.rotate(gf.components.spiral_inner_io_fiber_single, 90))
spiral_te.plot()

In [ ]:
# which is equivalent to
spiral_te = gf.compose(gf.routing.add_fiber_single, gf.functions.rotate90, gf.components.spiral_inner_io_fiber_single)
c = spiral_te(length=10e3)
c.plot()

In [ ]:
import gdsfactory as gf

spiral_te = gf.compose(gf.routing.add_fiber_single, gf.functions.rotate90, gf.components.spiral_inner_io_fiber_single)
sweep = [spiral_te(length=length) for length in [10e3, 20e3, 30e3]]
m = gf.pack(sweep)
c = m[0]
c.plot()

You can also add a `prefix` to each text label. For example `S` for the spirals at the `north-center`

`text_rectangular` is DRC clean and is anchored on `nc` (north-center)

In [ ]:
text_metal3 = gf.partial(gf.components.text_rectangular_multi_layer, layers=(gf.LAYER.M3,))

m = gf.pack(sweep, text=text_metal3, text_anchors=('nc',), text_prefix='s')
c = m[0]
c.plot()

In [ ]:
text_metal2 = gf.partial(gf.c.text, layer=gf.LAYER.M2)

m = gf.pack(sweep, text=text_metal2, text_anchors=('nc',), text_prefix='s')
c = m[0]
c.plot()

## Grid

In [ ]:
g = gf.grid(sweep)
g.plot()

In [ ]:
gh = gf.grid(sweep, shape=(1, len(sweep)))
gh.plot()

In [ ]:
ghymin = gf.grid(sweep, shape=(1, len(sweep)), align_y='ymin')
ghymin.plot()

You can also add text labels to each element of the sweep

In [ ]:
ghymin = gf.grid_with_text(sweep, shape=(1, len(sweep)), align_y='ymin', text=text_metal3)
ghymin.plot()

## Mask

You can easily define a mask using `grid` and `pack`

In [ ]:
import gdsfactory as gf

text_metal3 = gf.partial(gf.c.text_rectangular_multi_layer, layers=(gf.LAYER.M3,))
grid = gf.partial(gf.grid_with_text, text=text_metal3)
pack = gf.partial(gf.pack, text=text_metal3)

gratings_sweep = [gf.c.grating_coupler_elliptical(taper_angle=taper_angle) for taper_angle in [20, 30, 40]]
gratings = grid(gratings_sweep, text=None)
gratings.plot()

In [ ]:
gratings_sweep = [gf.c.grating_coupler_elliptical(taper_angle=taper_angle) for taper_angle in [20, 30, 40]]
gratings_loss_sweep = [gf.c.grating_coupler_loss_fiber_single(grating_coupler=grating) for grating in gratings_sweep]
gratings = grid(gratings_loss_sweep, shape=(1, len(gratings_loss_sweep)), spacing = (40,0))
gratings.plot()

In [ ]:
sweep_resistance = [gf.components.resistance_sheet(width=width) for width in [1, 10, 100]]
resistance = gf.pack(sweep_resistance)[0]
resistance.plot()

In [ ]:
spiral_te = gf.compose(gf.routing.add_fiber_single, gf.functions.rotate90, gf.components.spiral_inner_io_fiber_single)
sweep_spirals = [spiral_te(length=length) for length in [10e3, 20e3, 30e3]]
spirals = gf.pack(sweep_spirals)[0]
spirals.plot()

In [ ]:
mask = gf.pack([spirals, resistance, gratings])[0]
mask.plot()

As you can see you can define your mask in a single line.

For more complex mask, you can also create a new cell to build up more complexity

In [ ]:
@gf.cell
def mask():
    c = gf.Component()
    c << gf.pack([spirals, resistance, gratings])[0]
    c << gf.c.seal_ring(c.bbox)
    return c

c = mask(cache=False)
c.plot()

In [ ]:
gdspath = c.write_gds_with_metadata(gdsdir='extra')

In [ ]:
yaml_path = gdspath.with_suffix('.yml')

In [ ]:
labels_path = gf.mask.write_labels(gdspath=gdspath, layer_label=(201, 0))

In [ ]:
from omegaconf import OmegaConf

mask_metadata = OmegaConf.load(yaml_path)

In [ ]:
test_metadata = tm = gf.mask.merge_test_metadata(labels_path=labels_path, mask_metadata=mask_metadata)

In [ ]:
tm.keys()

```

CSV labels  ------|
                  |--> merge_test_metadata dict
                  |
YAML metatada  ---

```

In [ ]:
tm.spiral_inner_io_6dc6250a.full.length

In [ ]:
spiral_names = [s for s in test_metadata.keys() if s.startswith('spiral')]
spiral_names

In [ ]:
spiral_lengths = [test_metadata[spiral_name].length for spiral_name in spiral_names]
spiral_lengths

In [ ]:
gc_names = [s for s in test_metadata.keys() if s.startswith('grating')]
gc_names

In [ ]:
gc_taper_angles = [test_metadata[name].full.taper_angle for name in gc_names]
gc_taper_angles